In [9]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [18]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_20Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [11]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [12]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [13]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, _= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt':_,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [14]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [17]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response, payload_data['prompt']
    except Exception as e:
        print(e)
        

In [ ]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
#     prompts = read_payloads(request_payload_file)
    
    global df
    for i in range(20):
        prompts = read_payloads(request_payload_file)
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-7b84f7d6444e40c98bfdf60bd202d9fe","object":"text_completion","created":449358,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 4.281581 seconds 


Prompt==> Problem: Write a function in python to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s. 
 {"id":"cmpl-7ba6d2ac616a449ea8ee921c8e008c17","object":"text_completion","created":449358,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/c

Prompt==> Problem: Write a function in python to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-15c0e71428fd43fc9f303fb946db6695","object":"text_completion","created":449358,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nFibonacci sequence: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, ...\n\nThe Fibonacci sequence is defined as follows:\n\nF(0) = 0, F(1) = 1\n\nFor n > 1, F(n) = F(n-1) + F(n-2)\n\nHere is a Python function that generates the nth Fibonacci number:\n\n```python\ndef fibonacci(n):\n    if n <= 0:\n        return \"Input should be a positive integer.\"\n    elif n == 1:\n        return 0\n    elif n == 2:\n        return 1\n    else:\n        a, b = 0, 1\n       ","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":43,"total_tokens":243,"completion_tokens":200}}Prompt==> Problem: Write a 

Prompt==> Problem: Write a function in c++ to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-0afab517197c4d9b8147fb4c094d6b14","object":"text_completion","created":449362,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\n```cpp\n#include <iostream>\nusing namespace std;\n\nint factorial(int n) {\n    if(n == 0)\n        return 1;\n    else\n        return n * factorial(n - 1);\n}\n\nint main() {\n    int n;\n    cout << \"Enter a non-negative integer: \";\n    cin >> n;\n    cout << \"Factorial of \" << n << \" is \" << factorial(n) << endl;\n    return 0;\n}\n```\n\nThis program uses a recursive function to calculate the factorial of a given non-negative integer. The base case of the recursion is when n is 0, in which case the function returns 1. For all other values of n, the function ca

Prompt==> Problem: Write a function in Ruby to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-9af75e14c3d84261bd978e969d0cb165","object":"text_completion","created":449376,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef palindrome?(s)\n  s == s.reverse\nend\n\nputs palindrome?(\"racecar\") # true\nputs palindrome?(\"hello\") # false","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":40,"total_tokens":89,"completion_tokens":49}}
Execution time: 3.748835 seconds 


Prompt==> Problem: Write a function in c++ to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-f0d80f8c77c742ef94d622d99853320e","object":"text_completion","created":449366,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e11

Prompt==> Problem: Write a function in Ruby to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num. 
 {"id":"cmpl-b42446dc79f2449f9d68d3476ceb8fb1","object":"text_completion","created":449376,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef sum_of_digits(num)\n  num.to_s.chars.map(&:to_i).sum\nend\n\nputs sum_of_digits(1234) # Output: 10\nputs sum_of_digits(5678) # Output: 26\nputs sum_of_digits(9012) # Output: 4\nputs sum_of_digits(1000) # Output: 1\nputs sum_of_digits(1111) # Output: 4","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":44,"total_tokens":182,"completion_tokens":138}}
Execution time: 10.194699 seconds 


Prompt==> Problem: Write a function in javascript to check if a given number is a prime number.Input: Positive integer num.Output: Boolean, indicating whether num is prime.

...

Prompt==> Problem: Write a function in Golang to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-6fb1198a0195428b9dd5c6e8eb881098","object":"text_completion","created":449450,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nFibonacci sequence: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, ...\n\nHere is a simple function in Golang to generate the nth Fibonacci number:\n\n```go\npackage main\n\nimport \"fmt\"\n\nfunc fibonacci(n int) int {\n    if n <= 1 {\n        return n\n    }\n    a, b := 0, 1\n    for i := 2; i <= n; i++ {\n        a, b = b, a+b\n    }\n    return b\n}\n\nfunc main() {\n    fmt.Println(fibonacci(10))  // Output: 55\n}\n```\n","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":45,"total_tokens":245,"completion_tokens":200}}
Execution time: 14.177257 seconds 


Prompt==> Problem: Writ

Prompt==> Problem: Build a function in CSS to populate an HTML table with data from an array of objects.Input: Array of objects, each representing a row in the table.Output: None (update the HTML table). 
 {"id":"cmpl-d0e148bde4e04960a9bcd5a6c0bd397f","object":"text_completion","created":449455,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can do this in CSS:\n\n```css\n@keyframes populate-table {\n  0% {\n    opacity: 0;\n  }\n  100% {\n    opacity: 1;\n  }\n}\n\n.populate-table {\n  animation: populate-table 1s;\n}\n```\n\nIn this example, the `populate-table` class is added to the table when the data is ready to be displayed. The animation will gradually increase the opacity of the table from 0 to 1, making it visible.\n\nYou can use JavaScript or jQuery to add the class to the table when the data is ready:\n\n```javascript\nvar data = [\n  {name: '

Prompt==> Problem: Create an animated dropdown menu in react that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu). 
 {"id":"cmpl-40ee1540c3e34cedbb321cfef6f45645","object":"text_completion","created":449462,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can create an animated dropdown menu in React using CSS and JavaScript.\n\n```jsx\nimport React, { useState } from 'react';\n\nconst DropdownMenu = () => {\n  const [isOpen, setIsOpen] = useState(false);\n\n  const toggleDropdown = () => {\n    setIsOpen(!isOpen);\n  };\n\n  return (\n    <div className=\"dropdown\">\n      <button onClick={toggleDropdown}>\n        {isOpen ? 'Close' : 'Open'} Dropdown\n      </button>\n      {isOpen && (\n        <div className=\"dropdown-menu\">\n          <ul>\n            <li>Opt

Prompt==> Problem: Implement an autocomplete feature in angular for a search input field.Input: String, user input in the search field.Output: None (update the autocomplete suggestions). 
 {"id":"cmpl-01f9f8da3767499c9f71cd95acebcd5c","object":"text_completion","created":449468,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\nHere is a simple example of how you can implement an autocomplete feature in Angular. This example uses a static list of items for simplicity, but in a real-world application, you would likely fetch the data from a server or database.\n\nFirst, you need to create a component for the autocomplete feature. Let's call it `autocomplete.component.ts`:\n\n```typescript\nimport { Component, OnInit } from '@angular/core';\n\n@Component({\n  selector: 'app-autocomplete',\n  templateUrl: './autocomplete.component.html',\n  styleUrls: ['./autocomplete.component.css']

Prompt==> Problem: Write a function in angular that dynamically creates an HTML element and appends it to the document.Input: String, element type (e.g., 'div', 'button').Output: None (update the HTML document). 
 {"id":"cmpl-34c0f79df21e492793695a70718eef11","object":"text_completion","created":449472,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple function in Angular that creates an HTML element and appends it to the document:\n\n```typescript\nimport { Component, ElementRef, Renderer2 } from '@angular/core';\n\n@Component({\n  selector: 'app-root',\n  templateUrl: './app.component.html',\n  styleUrls: ['./app.component.css']\n})\nexport class AppComponent {\n  constructor(private el: ElementRef, private renderer: Renderer2) {}\n\n  createElement(elementType: string, text: string) {\n    const newElement = this.renderer.createElement(elementType);\n    const textNode = t

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))